# Traffic Violation RAG System
In this exam, you will implement a Retrieval-Augmented Generation (RAG) system that uses a language model and a vector database to answer questions about traffic violations. The goal is to generate answers with relevant data based on a dataset of traffic violations and fines.

Here are helpful resources:
* [LangChain](https://www.langchain.com/)
* [groq cloud documentation](https://console.groq.com/docs/models)
* [LangChain HuggingFace](https://python.langchain.com/docs/integrations/text_embedding/sentence_transformers/)
* [Chroma Vector Store](https://python.langchain.com/docs/integrations/vectorstores/chroma/)
* [Chroma Website](https://docs.trychroma.com/getting-started)
* [ChatGroq LangChain](https://python.langchain.com/docs/integrations/chat/groq/)
* [LLM Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.llm.LLMChain.html#langchain.chains.llm.LLMChain)

Dataset [source](https://www.moi.gov.sa/wps/portal/Home/sectors/publicsecurity/traffic/contents/!ut/p/z0/04_Sj9CPykssy0xPLMnMz0vMAfIjo8ziDTxNTDwMTYy83V0CTQ0cA71d_T1djI0MXA30gxOL9L30o_ArApqSmVVYGOWoH5Wcn1eSWlGiH1FSlJiWlpmsagBlKCQWqRrkJmbmqRqUZebngB2gUJAKdERJZmqxfkG2ezgAhzhSyw!!/)

Some installs if needed:
```python
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq LLMChain
```

In [1]:
!kaggle datasets download -d khaledzsa/dataset
!unzip dataset.zip

Dataset URL: https://www.kaggle.com/datasets/khaledzsa/dataset
License(s): unknown
  0% 0.00/3.73k [00:00<?, ?B/s]
100% 3.73k/3.73k [00:00<00:00, 8.04MB/s]
Archive:  dataset.zip
  inflating: Dataset.csv             


## Step 1: Install Required Libraries

To begin, install the necessary libraries for this project. The libraries include `LangChain` for building language model chains, and `Chroma` for managing a vector database.

In [2]:
!pip install langchain_huggingface langchain langchain-community langchain_chroma Chroma langchain_groq

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Usi

In [3]:
import pandas as pd
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings.sentence_transformer import (
    SentenceTransformerEmbeddings,
)
from langchain.prompts import PromptTemplate
import getpass
from langchain_community.vectorstores import Chroma
import markdown
from langchain_groq import ChatGroq
from langchain.chains import LLMChain

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


# Step 2: Load the Traffic Violations Dataset

You are provided with a dataset of traffic violations. Load the CSV file into a pandas DataFrame and preview the first few rows of the dataset using `.head()`. You can also try and see the dataset's characteristics.

In [4]:
df = pd.read_csv('/content/Dataset.csv')
df.head()

,المخالفة,الغرامة
0,قيادة المركبة في الأسواق التي لا يسمح بالقيادة...,الغرامة المالية 100 - 150 ريال
1,ترك المركبة مفتوحة وفي وضع التشغيل بعد مغادرتها.,الغرامة المالية 100 - 150 ريال
2,عدم وجود تأمين ساري للمركبة.,الغرامة المالية 100 - 150 ريال
3,عبور المشاة للطرق من غير الأماكن المخصصة لهم.,الغرامة المالية 100 - 150 ريال
4,عدم تقيد المشاة بالإشارات الخاصة بهم.,الغرامة المالية 100 - 150 ريال


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   المخالفة  104 non-null    object
 1   الغرامة   104 non-null    object
dtypes: object(2)
memory usage: 1.8+ KB


In [7]:
df.sample(5)

,المخالفة,الغرامة
52,استخدام السائق بيده أي جهاز محمول أثناء سير ال...,الغرامة المالية 500 - 900 ريال
15,عدم ربط حزام الأمان.,الغرامة المالية 150 - 300 ريال
19,التجمهر في موقع الحادث.,الغرامة المالية 150 - 300 ريال
85,تجاوز حافلات النقل المدرسي عند توقفها للتحميل ...,الغرامة المالية 3000 - 6000 ريال
84,تجاوز إشارة المرور الضوئية أثناء الضوء الأحمر.,الغرامة المالية 3000 - 6000 ريال


## Step 3: Create Markdown Content from the Dataset

For each traffic violation in the dataset, you will generate markdown text that describes the violation and the associated fine. Create a loop to iterate through the dataset and store the generated markdown in a list. Each fine should look like this:

**المخالفة** - الغرامة

In [8]:
directory = 'data/markdown_files'
os.makedirs(directory, exist_ok=True)


for i in range(0, 103):
    title = df['المخالفة'].iloc[i]
    content = df['الغرامة'].iloc[i]

    markdown_content = f"# {title}\n\n"
    markdown_content += f"{content}\n\n"

    with open(f'{directory}/{i}.md', 'w', encoding='utf-8') as file:
        file.write(markdown_content)

In [9]:
# Store the generated markdown in a list
markdown_texts = []
for filename in os.listdir(directory):
  if filename.endswith(".md"):
    with open(os.path.join(directory, filename), 'r', encoding='utf-8') as file:
      markdown_content = file.read()
      html_content = markdown.markdown(markdown_content)
      markdown_texts.append(html_content)

## Step 4: Chunk the Markdown Data

Using LangChain's `RecursiveCharacterTextSplitter`, split the markdown texts into smaller chunks that will be stored in the vector database.

In [10]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size= 100,
    chunk_overlap= 20,
    length_function=len,
    is_separator_regex=False
)

documents = splitter.create_documents(markdown_texts)

## Step 5: Generate Embeddings for the Documents

Generate embeddings for the chunks of text using HuggingFace's pre-trained Arabic language model. These embeddings will be stored in a `Chroma` vector store.

In [11]:
embeddings = SentenceTransformerEmbeddings(
    model_name="asafaya/bert-base-arabic"
)

db = Chroma.from_documents(
    documents,
    embeddings,
    persist_directory="./chroma_db"
)

<ipython-input-11-75e942d302ad>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding_function = SentenceTransformerEmbeddings(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens)

config.json:   0%|          | 0.00/491 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/334k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

# Step 6: Define the RAG Prompt Template

Define a custom prompt template in Arabic to retrieve traffic violation-related answers based on the context. Ensure the template encourages the model to give **advice** in **Arabic**, staying within the context provided.

In [12]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:
Context: {context}
Question: {question}
Your answer:"""

In [13]:
prompt_template = PromptTemplate(
    template=PROMPT_TEMPLATE, input_variables=["context", "question"]
)

## Step 7: Initialize the Language Model

Initialize the language model using the Groq API. Set up the model with a specific configuration, including the API key, temperature setting, and model name.

In [ ]:
groq_api_key =os.environ["GROQ_API"] = getpass.getpass()

llm = ChatGroq(
    temperature=0.4,
    groq_api_key=groq_api_key,
    model_name="llama3-8b-8192"
)

··········


## Step 8: Create the LLM Chain

Now, you will create an LLM Chain that combines the language model and the prompt template you defined. This chain will be used to generate responses based on the retrieved context.

In [ ]:
MODEL = LLMChain(llm=llm,
                 prompt=prompt_template,
                 verbose=True)

## Step 9: Implement the Query Function

Create a function `query_rag` that will take a user query as input, retrieve relevant context from the vector store, and use the language model to generate a response based on that context.

In [ ]:
def query_rag(query: str):
    similarity_search_results = db.similarity_search_with_score(query, k=4)
    context_text = "\n\n".join([doc.page_content for doc, _score in similarity_search_results])

    rag_response = MODEL.invoke({"context": context_text, "question": query})

    return rag_response

In [ ]:
response = query_rag("ما الذي يترتب على عدم تقيد المشاة بالإشارات الخاصة بهم؟")
print(response)

Prompt after formatting:

Answer the question based only on the following context:
Context: <h1>عدم إعطاء أفضلية المرور لوسائل النقل العام كالقطارات أو الحافلات وما في حكمها في حال سيرها على

<h1>تظليل زجاج المركبة دون التقيد بالضوابط التي تضعها الإدارة المختصة.</h1>

<h1>عدم قيام السائق في حال تغيير اتجاهه بالدوران للخلف بإعطاء أفضلية المرور للمركبات القادمة من

<h1>عدم الوقوف عند مراكز الضبط الأمني أو نقاط التفتيش أو عدم الوقوف للدورية الأمنية عند وجود توجيه
Question: ما الذي يترتب على عدم تقيد المشاة بالإشارات الخاصة بهم؟
Your answer:


> Finished chain.
{'context': '<h1>عدم إعطاء أفضلية المرور لوسائل النقل العام كالقطارات أو الحافلات وما في حكمها في حال سيرها على\n\n<h1>تظليل زجاج المركبة دون التقيد بالضوابط التي تضعها الإدارة المختصة.</h1>\n\n<h1>عدم قيام السائق في حال تغيير اتجاهه بالدوران للخلف بإعطاء أفضلية المرور للمركبات القادمة من\n\n<h1>عدم الوقوف عند مراكز الضبط الأمني أو نقاط التفتيش أو عدم الوقوف للدورية الأمنية عند وجود توجيه', 'question': 'ما الذي يترتب على عدم تقيد ال

## Step 10: Inference - Running Queries in the RAG System

In this final step, you will implement an inference pipeline to handle real-time queries. You will allow the system to retrieve the most relevant violations and fines based on a user's input and generate a response.

1. Inference Workflow:

  * The user inputs a query (e.g., "ماهي الغرامة على القيادة بدون رخصة؟").
  * The system searches for the most relevant context from the traffic violation vector store.
  * It generates an answer and advice based on the context.

2. Goal:
  * Run the inference to answer questions based on the traffic violation dataset.

In [ ]:
def inference_pipeline(user_query: str):
    response = query_rag(user_query)
    return response

In [ ]:
user_input = "ماهي الغرامة على القيادة بدون رخصة؟"
print(inference_pipeline(user_input))

Prompt after formatting:

Answer the question based only on the following context:
Context: الأساسية دون اتخاذ الإجراءات النظامية.</h1>

الآداب العامة أثناء القيادة.</h1>

<h1>عدم إعطاء أفضلية المرور لوسائل النقل العام كالقطارات أو الحافلات وما في حكمها في حال سيرها على

<h1>عدم قيام السائق في حال تغيير اتجاهه بالدوران للخلف بإعطاء أفضلية المرور للمركبات القادمة من
Question: ماهي الغرامة على القيادة بدون رخصة؟
Your answer:


> Finished chain.
{'context': 'الأساسية دون اتخاذ الإجراءات النظامية.</h1>\n\nالآداب العامة أثناء القيادة.</h1>\n\n<h1>عدم إعطاء أفضلية المرور لوسائل النقل العام كالقطارات أو الحافلات وما في حكمها في حال سيرها على\n\n<h1>عدم قيام السائق في حال تغيير اتجاهه بالدوران للخلف بإعطاء أفضلية المرور للمركبات القادمة من', 'question': 'ماهي الغرامة على القيادة بدون رخصة؟', 'text': 'Based on the provided context, there is no mention of a fine for driving without a license. The context only discusses traffic rules and regulations, but does not mention the specific topic of dri

### The Groq It does not support the Arabic language.